In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import Tensor

import numpy as np
import tensorflow as tf
from scipy.misc import imread, imresize, imsave, imshow
from imagenet_classes import class_names 
import matplotlib.pyplot as plt
import json
import pickle
import os
import re



%matplotlib inline

# Import

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#  load fine grained classifcation dataset from memory

In [3]:
meta_1 = json.load(open("/home/tuna/Projects/CS601R/Project3/lab10/lib/cub_metadata.json", "r"))
meta_2 = pickle.load(open("/home/tuna/Projects/CS601R/Project3/lab10/lib/cub_metadata.pkl", "r"))

In [4]:
#  remove number prefixes from names
for inst in meta_2['class_names']:
    meta_2['class_names'][inst] = meta_2['class_names'][inst][4:].lower()
    

In [5]:
meta_2['class_names'][1]

'black_footed_albatross'

In [6]:
#  clean up imageNet classes for comparison
standardized_class_names = []
for name in class_names:
    standardized_class_names.append("_".join(re.split("[^a-zA-Z\d:]+", name.lower())))
    


In [7]:
rootdir = '/home/tuna/Projects/CS601R/Project3/lab10/lib/images_sq224'

all_ims_original = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        all_ims_original.append( imresize(imread(os.path.join(subdir, file), mode='RGB'), (224, 224)))
    
all_ims_original = np.asarray(all_ims_original)

In [8]:
#  create training/test sets

test_data_original = []
test_classes_original = []

# for i in range(1, len(meta_2['train_test_split']['testing'])):
for i in range(1, 5):
    test_data_original.extend(all_ims_original[meta_2['train_test_split']['testing'][i]])
    test_classes_original.extend([i] * len(meta_2['train_test_split']['testing'][i])) 
    
    
    
train_data_original = []
train_classes_original = []

# for i in range(1, len(meta_2['train_test_split']['training'])):
for i in range(1, 5):
    train_data_original.extend(all_ims_original[meta_2['train_test_split']['training'][i]])
    train_classes_original.extend([i] * len(meta_2['train_test_split']['training'][i]))   

In [9]:
#  get class names
def get_class_names(passed_indexes):
    class_names = []
    for i in passed_indexes:
        class_names.append(meta_2['class_names'][i])
    
    return class_names

test_classes_original_names = get_class_names(test_classes_original)
train_classes_original_names = get_class_names(train_classes_original)

In [10]:
#  one hot encoded versions

train_classes_ohe = np.zeros((len(train_classes_original),5))
for i,c in enumerate(train_classes_original):
    train_classes_ohe[i,c] = 1.0
    
test_classes_ohe = np.zeros((len(test_classes_original),5))
for i,c in enumerate(test_classes_original):
    test_classes_ohe[i,c] = 1.0
    
    
train_classes_ohe[0]

array([ 0.,  1.,  0.,  0.,  0.])

In [11]:
    
"""Define get next batch method"""
def get_next_batch(data, classes, num_to_select):
  #  shuffle with labels
  data_shuffled_mat =  np.asarray(zip(data, classes))
  np.random.shuffle(data_shuffled_mat)
  
  data_mat_selected = np.asarray(list(data_shuffled_mat[:num_to_select,0]))
  data_classes_selected = np.asarray(list(data_shuffled_mat[:num_to_select,1]))
  
  return data_mat_selected.astype(float), data_classes_selected.astype(float)



In [12]:
tmp_test_batch_data,tmp_test_batch_labels  = get_next_batch(train_data_original, train_classes_ohe, 2)
tmp_test_batch_labels

array([[ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.]])

In [13]:
train_data_original[0].shape

(224, 224, 3)

In [14]:
train_classes_original_names

['black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'black_footed_albatross',
 'laysan_albatross',
 'laysan_albatross',
 'laysan_albatross',
 'laysan_albatross',
 'laysan_albatross',
 'laysan_albatross',
 'laysan_albatross',
 'laysan_albatross',
 'laysan_albatross',
 

In [15]:
train_classes_ohe


array([[ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  

# Model

#  CNN model MNIST

In [3]:
sess = tf.InteractiveSession()    
   
batch_size = 50
    

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def max_pool_4x4(x):
  return tf.nn.max_pool(x, ksize=[1, 4, 4, 1],
                        strides=[1, 4, 4, 1], padding='SAME')

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')    
    
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=1e-4)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)








with tf.name_scope('Wx_B') as scope:

    x = tf.placeholder(tf.float32, shape=[None, 784])
    x_batch_size = tf.shape(x)[0]
    y_ = tf.placeholder(tf.float32, shape=[None, 10])
    x_image = tf.reshape(x, [-1,28,28,1])

    W_conv1_bnn1 = weight_variable([5, 5, 1, 32])
    b_conv1_bnn1 = bias_variable([32])
    h_conv1_bnn1 = tf.nn.relu(conv2d(x_image, W_conv1_bnn1) + b_conv1_bnn1)
    h_pool1_bnn1 = max_pool_2x2(h_conv1_bnn1)

    W_conv2_bnn1 = weight_variable([5, 5, 32, 40])
    b_conv2_bnn1 = bias_variable([40])
    h_conv2_bnn1 = tf.nn.relu(conv2d(h_pool1_bnn1, W_conv2_bnn1) + b_conv2_bnn1)
    h_pool2_bnn1 = max_pool_2x2(h_conv2_bnn1)

    W_conv3_bnn1 = weight_variable([5, 5, 40, 16])
    b_conv3_bnn1 = bias_variable([16])
    h_conv3_bnn1 = tf.nn.relu(conv2d(h_pool2_bnn1, W_conv3_bnn1) + b_conv3_bnn1)
    #     h_pool3 = max_pool_2x2(h_conv3)

    W_fc1_bnn1 = weight_variable([7 * 7 * 16, 100])
    b_fc1_bnn1 = bias_variable([100])
    h_conv3_flat_bnn1 = tf.reshape(h_conv3_bnn1, [-1, 7*7*16])
    h_fc1_bnn1 = tf.nn.relu(tf.matmul(h_conv3_flat_bnn1, W_fc1_bnn1) + b_fc1_bnn1)


    keep_prob = tf.placeholder(tf.float32)
    y_raw_bnn1 = h_fc1_bnn1

        
        
    
    
    #  final fc layer transformation to 10 classes
    W_fc3 = weight_variable([100, 10])
    b_fc3 = bias_variable([10])
    y_fc3 = tf.matmul(y_raw_bnn1, W_fc3) + b_fc3

    
    y_conv=tf.nn.softmax(y_fc3)
    out_shape = tf.shape(y_conv)
    print(y_conv.shape)

    



    
    
def l2_loss(tensor_inst):
    return tf.reduce_sum(tensor_inst ** 2)
beta = 0.0001

with tf.name_scope('cost') as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1])) + \
                beta*l2_loss(W_conv1_bnn1) +\
                beta*l2_loss(b_conv1_bnn1) +\
                beta*l2_loss(W_conv2_bnn1) +\
                beta*l2_loss(b_conv2_bnn1) +\
                beta*l2_loss(W_conv3_bnn1) +\
                beta*l2_loss(b_conv3_bnn1) +\
                beta*l2_loss(W_fc1_bnn1) +\
                beta*l2_loss(b_fc1_bnn1) +\
                beta*l2_loss(W_fc3) +\
                beta*l2_loss(b_fc3)



with tf.name_scope('optimizer') as scope:
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)



with tf.name_scope('accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





#  summary writers for debugging
summary_writer = tf.summary.FileWriter( '/home/tuna/Projects/CS678/Deep_Learning_Project_2/tf_logs/vanilla_cnn_mnist_2', graph=sess.graph )
acc_summary = tf.summary.scalar( 'accuracy', accuracy )
loss_summary = tf.summary.scalar( 'loss', cross_entropy )
merged_summary_op = tf.summary.merge_all()

sess.run(tf.global_variables_initializer())






for i in range(10000):
  batch = mnist.train.next_batch(batch_size)
#   print("Batch", i)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
#   summary, _, outshape, wc111, bc111, yop, yr, yopv, yc, hp1, hp2 = sess.run([merged_summary_op,train_step,out_shape, W_conv1, b_conv1, y_outer_product, y_raw, y_outer_product_vec, y_conv, h_pool1, h_pool2], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  summary, _, outshape, wc111, bc111, yc, hp1 = sess.run([merged_summary_op,train_step,out_shape, W_conv1_bnn1, b_conv1_bnn1,  y_conv, h_pool1_bnn1], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  summary_writer.add_summary(summary, i)
#   print("out shape", outshape)
#   print("wc1", wc111)
#   print("bc1", bc111)
#   print("hp1", hp1)
#   print("hp2", hp2)
#   print("yr", yr)
#   print("yop", yop)
#   print("yopv", yopv)
#   print("yc", yc)
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))



summary_writer.close()
sess.close()

(?, 10)
step 0, training accuracy 0.1
step 100, training accuracy 0.12
step 200, training accuracy 0.16
step 300, training accuracy 0.14
step 400, training accuracy 0.12
step 500, training accuracy 0.16
step 600, training accuracy 0.16
step 700, training accuracy 0.28
step 800, training accuracy 0.22
step 900, training accuracy 0.32
step 1000, training accuracy 0.34
step 1100, training accuracy 0.48
step 1200, training accuracy 0.72
step 1300, training accuracy 0.74
step 1400, training accuracy 0.68
step 1500, training accuracy 0.7
step 1600, training accuracy 0.86
step 1700, training accuracy 0.68
step 1800, training accuracy 0.82
step 1900, training accuracy 0.84
step 2000, training accuracy 0.7
step 2100, training accuracy 0.8
step 2200, training accuracy 0.82
step 2300, training accuracy 0.84
step 2400, training accuracy 0.84
step 2500, training accuracy 0.84
step 2600, training accuracy 0.86
step 2700, training accuracy 0.88
step 2800, training accuracy 0.78
step 2900, training ac

#  CNN model BIRDS

In [16]:
sess = tf.InteractiveSession()    
   
batch_size = 50
    

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def max_pool_4x4(x):
  return tf.nn.max_pool(x, ksize=[1, 4, 4, 1],
                        strides=[1, 4, 4, 1], padding='SAME')

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')    
    
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=1e-4)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)








with tf.name_scope('Wx_B') as scope:

    x = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
    x_batch_size = tf.shape(x)[0]
    y_ = tf.placeholder(tf.float32, shape=[None, 5])
    x_image = tf.reshape(x, [-1,224, 224, 3])

    W_conv1_bnn1 = weight_variable([5, 5, 3, 32])
    b_conv1_bnn1 = bias_variable([32])
    h_conv1_bnn1 = tf.nn.relu(conv2d(x_image, W_conv1_bnn1) + b_conv1_bnn1)
    h_pool1_bnn1 = max_pool_2x2(h_conv1_bnn1)

    W_conv2_bnn1 = weight_variable([5, 5, 32, 40])
    b_conv2_bnn1 = bias_variable([40])
    h_conv2_bnn1 = tf.nn.relu(conv2d(h_pool1_bnn1, W_conv2_bnn1) + b_conv2_bnn1)
    h_pool2_bnn1 = max_pool_2x2(h_conv2_bnn1)

    W_conv3_bnn1 = weight_variable([5, 5, 40, 16])
    b_conv3_bnn1 = bias_variable([16])
    h_conv3_bnn1 = tf.nn.relu(conv2d(h_pool2_bnn1, W_conv3_bnn1) + b_conv3_bnn1)
    #     h_pool3 = max_pool_2x2(h_conv3)

    W_fc1_bnn1 = weight_variable([56 * 56 * 16, 100])
    b_fc1_bnn1 = bias_variable([100])
    h_conv3_flat_bnn1 = tf.reshape(h_conv3_bnn1, [-1, 56 * 56 * 16])
    h_fc1_bnn1 = tf.nn.relu(tf.matmul(h_conv3_flat_bnn1, W_fc1_bnn1) + b_fc1_bnn1)


    keep_prob = tf.placeholder(tf.float32)
    y_raw_bnn1 = h_fc1_bnn1

        
        
    
    
    #  final fc layer transformation to 5 classes
    W_fc3 = weight_variable([100, 5])
    b_fc3 = bias_variable([5])
    y_fc3 = tf.matmul(y_raw_bnn1, W_fc3) + b_fc3

    
    y_conv=tf.nn.softmax(y_fc3)
    out_shape = tf.shape(y_conv)
    print(y_conv.shape)

    



    
    
def l2_loss(tensor_inst):
    return tf.reduce_sum(tensor_inst ** 2)
beta = 0.0001

with tf.name_scope('cost') as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1])) + \
                beta*l2_loss(W_conv1_bnn1) +\
                beta*l2_loss(b_conv1_bnn1) +\
                beta*l2_loss(W_conv2_bnn1) +\
                beta*l2_loss(b_conv2_bnn1) +\
                beta*l2_loss(W_conv3_bnn1) +\
                beta*l2_loss(b_conv3_bnn1) +\
                beta*l2_loss(W_fc1_bnn1) +\
                beta*l2_loss(b_fc1_bnn1) +\
                beta*l2_loss(W_fc3) +\
                beta*l2_loss(b_fc3)



with tf.name_scope('optimizer') as scope:
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)



with tf.name_scope('accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





#  summary writers for debugging
summary_writer = tf.summary.FileWriter( '/home/tuna/Projects/CS678/Deep_Learning_Project_2/tf_logs/vanilla_cnn_birds_3', graph=sess.graph )
acc_summary = tf.summary.scalar( 'accuracy', accuracy )
loss_summary = tf.summary.scalar( 'loss', cross_entropy )
merged_summary_op = tf.summary.merge_all()

sess.run(tf.global_variables_initializer())






for i in range(100):
#   batch = mnist.train.next_batch(batch_size)
  batch_xs, batch_ys = get_next_batch(train_data_original, train_classes_ohe, batch_size)

    
    
#   print("Batch", i)
  if i%1 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch_xs, y_: batch_ys, keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
#   summary, _, outshape, wc111, bc111, yop, yr, yopv, yc, hp1, hp2 = sess.run([merged_summary_op,train_step,out_shape, W_conv1, b_conv1, y_outer_product, y_raw, y_outer_product_vec, y_conv, h_pool1, h_pool2], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  summary, _, outshape, wc111, bc111, yc, hp1 = sess.run([merged_summary_op,train_step,out_shape, W_conv1_bnn1, b_conv1_bnn1,  y_conv, h_pool1_bnn1], feed_dict={x:batch_xs, y_: batch_ys, keep_prob: 0.5})
  summary_writer.add_summary(summary, i)
#   print("out shape", outshape)
#   print("wc1", wc111)
#   print("bc1", bc111)
#   print("hp1", hp1)
#   print("hp2", hp2)
#   print("yr", yr)
#   print("yop", yop)
#   print("yopv", yopv)
#   print("yc", yc)
print("test accuracy %g"%accuracy.eval(feed_dict={
    x:test_data_original, y_: test_classes_ohe, keep_prob: 1.0}))





summary_writer.close()
sess.close()

(?, 5)
step 0, training accuracy 0.2
step 1, training accuracy 0.34
step 2, training accuracy 0.24
step 3, training accuracy 0.26
step 4, training accuracy 0.28
step 5, training accuracy 0.24
step 6, training accuracy 0.28
step 7, training accuracy 0.3
step 8, training accuracy 0.18
step 9, training accuracy 0.3
step 10, training accuracy 0.2
step 11, training accuracy 0.22
step 12, training accuracy 0.32
step 13, training accuracy 0.32
step 14, training accuracy 0.2
step 15, training accuracy 0.22
step 16, training accuracy 0.28
step 17, training accuracy 0.22
step 18, training accuracy 0.24
step 19, training accuracy 0.1
step 20, training accuracy 0.28
step 21, training accuracy 0.3
step 22, training accuracy 0.22
step 23, training accuracy 0.24
step 24, training accuracy 0.22
step 25, training accuracy 0.22
step 26, training accuracy 0.2
step 27, training accuracy 0.22
step 28, training accuracy 0.26
step 29, training accuracy 0.32
step 30, training accuracy 0.24
step 31, training a

(?, 5)
step 0, training accuracy 0
step 1, training accuracy 0.22
step 2, training accuracy 0.28
step 3, training accuracy 0.22
step 4, training accuracy 0.22
step 5, training accuracy 0.24
step 6, training accuracy 0.3
step 7, training accuracy 0.2
step 8, training accuracy 0.26
step 9, training accuracy 0.2
step 10, training accuracy 0.24
step 11, training accuracy 0.16
step 12, training accuracy 0.24
step 13, training accuracy 0.22
step 14, training accuracy 0.26
step 15, training accuracy 0.14
step 16, training accuracy 0.22
step 17, training accuracy 0.34
step 18, training accuracy 0.14
step 19, training accuracy 0.2
step 20, training accuracy 0.18
step 21, training accuracy 0.26
step 22, training accuracy 0.26
step 23, training accuracy 0.24
step 24, training accuracy 0.2
step 25, training accuracy 0.26
step 26, training accuracy 0.32
step 27, training accuracy 0.28
step 28, training accuracy 0.22
step 29, training accuracy 0.24
step 30, training accuracy 0.26
step 31, training a

NameError: name 'mnist' is not defined

#  Baseline Model MNIST

In [3]:
sess = tf.InteractiveSession()    
   
batch_size = 50
    

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=1e-4)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)








with tf.name_scope('Wx_B') as scope:

    x = tf.placeholder(tf.float32, shape=[None, 784])
    x_batch_size = tf.shape(x)[0]
    y_ = tf.placeholder(tf.float32, shape=[None, 10])
    x_image = tf.reshape(x, [-1,784])
        
        
    #  final fc layer transformation to 10 classes
    W_fc3 = weight_variable([784, 10])
    b_fc3 = bias_variable([10])
    y_fc3 = tf.matmul(x_image, W_fc3) + b_fc3

    
    y_conv=tf.nn.softmax(y_fc3)
    out_shape = tf.shape(y_conv)
    print(y_conv.shape)

    




with tf.name_scope('cost') as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))



with tf.name_scope('optimizer') as scope:
    train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_entropy)



with tf.name_scope('accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





#  summary writers for debugging
summary_writer = tf.summary.FileWriter( '/home/tuna/Projects/CS678/Deep_Learning_Project_2/tf_logs/baseline_mnist_1', graph=sess.graph )
acc_summary = tf.summary.scalar( 'accuracy', accuracy )
loss_summary = tf.summary.scalar( 'loss', cross_entropy )
merged_summary_op = tf.summary.merge_all()

sess.run(tf.global_variables_initializer())






for i in range(10000):
  batch = mnist.train.next_batch(batch_size)
#   print("Batch", i)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1]})
    print("step %d, training accuracy %g"%(i, train_accuracy))
#   summary, _, outshape, wc111, bc111, yop, yr, yopv, yc, hp1, hp2 = sess.run([merged_summary_op,train_step,out_shape, W_conv1, b_conv1, y_outer_product, y_raw, y_outer_product_vec, y_conv, h_pool1, h_pool2], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  summary, _ = sess.run([merged_summary_op,train_step], feed_dict={x: batch[0], y_: batch[1]})
  summary_writer.add_summary(summary, i)
#   print("out shape", outshape)
#   print("wc1", wc111)
#   print("bc1", bc111)
#   print("hp1", hp1)
#   print("hp2", hp2)
#   print("yr", yr)
#   print("yop", yop)
#   print("yopv", yopv)
#   print("yc", yc)
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels}))



summary_writer.close()
sess.close()

(?, 10)
step 0, training accuracy 0.14
step 100, training accuracy 0.72
step 200, training accuracy 0.78
step 300, training accuracy 0.5
step 400, training accuracy 0.78
step 500, training accuracy 0.62
step 600, training accuracy 0.78
step 700, training accuracy 0.7
step 800, training accuracy 0.66
step 900, training accuracy 0.64
step 1000, training accuracy 0.78
step 1100, training accuracy 0.78
step 1200, training accuracy 0.84
step 1300, training accuracy 0.64
step 1400, training accuracy 0.78
step 1500, training accuracy 0.58
step 1600, training accuracy 0.7
step 1700, training accuracy 0.76
step 1800, training accuracy 0.74
step 1900, training accuracy 0.82
step 2000, training accuracy 0.72
step 2100, training accuracy 0.74
step 2200, training accuracy 0.78
step 2300, training accuracy 0.74
step 2400, training accuracy 0.72
step 2500, training accuracy 0.68
step 2600, training accuracy 0.66
step 2700, training accuracy 0.68
step 2800, training accuracy 0.72
step 2900, training a

# Baseline Model with BIRDS

In [16]:
sess = tf.InteractiveSession()    
   
batch_size = 50
    

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=1e-4)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)








with tf.name_scope('Wx_B') as scope:

    x = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
    x_batch_size = tf.shape(x)[0]
    y_ = tf.placeholder(tf.float32, shape=[None, 5])
    x_image = tf.reshape(x, [-1, 224 * 224 * 3])

        
        
    
    
    #  final fc layer transformation to 5 classes
    W_fc3 = weight_variable([224 * 224 * 3, 5])
    b_fc3 = bias_variable([5])
    y_fc3 = tf.matmul(x_image, W_fc3) + b_fc3

    
    y_conv=tf.nn.softmax(y_fc3)
    out_shape = tf.shape(y_conv)
    print(y_conv.shape)

    



with tf.name_scope('cost') as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))


with tf.name_scope('optimizer') as scope:
    train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_entropy)



with tf.name_scope('accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





#  summary writers for debugging
summary_writer = tf.summary.FileWriter( '/home/tuna/Projects/CS678/Deep_Learning_Project_2/tf_logs/basline_birds_1', graph=sess.graph )
acc_summary = tf.summary.scalar( 'accuracy', accuracy )
loss_summary = tf.summary.scalar( 'loss', cross_entropy )
merged_summary_op = tf.summary.merge_all()

sess.run(tf.global_variables_initializer())






for i in range(1000):
#   batch = mnist.train.next_batch(batch_size)
  batch_xs, batch_ys = get_next_batch(train_data_original, train_classes_ohe, batch_size)

    
    
#   print("Batch", i)
  if i%1 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch_xs, y_: batch_ys})
    print("step %d, training accuracy %g"%(i, train_accuracy))
#   summary, _, outshape, wc111, bc111, yop, yr, yopv, yc, hp1, hp2 = sess.run([merged_summary_op,train_step,out_shape, W_conv1, b_conv1, y_outer_product, y_raw, y_outer_product_vec, y_conv, h_pool1, h_pool2], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  summary, _ = sess.run([merged_summary_op,train_step], feed_dict={x:batch_xs, y_: batch_ys})
  summary_writer.add_summary(summary, i)
#   print("out shape", outshape)
#   print("wc1", wc111)
#   print("bc1", bc111)
#   print("hp1", hp1)
#   print("hp2", hp2)
#   print("yr", yr)
#   print("yop", yop)
#   print("yopv", yopv)
#   print("yc", yc)
print("test accuracy %g"%accuracy.eval(feed_dict={
    x:test_data_original, y_: test_classes_ohe}))



summary_writer.close()
sess.close()

(?, 5)
step 0, training accuracy 0.24
step 1, training accuracy 0.24
step 2, training accuracy 0
step 3, training accuracy 0
step 4, training accuracy 0
step 5, training accuracy 0
step 6, training accuracy 0
step 7, training accuracy 0
step 8, training accuracy 0
step 9, training accuracy 0
step 10, training accuracy 0
step 11, training accuracy 0
step 12, training accuracy 0
step 13, training accuracy 0
step 14, training accuracy 0
step 15, training accuracy 0
step 16, training accuracy 0
step 17, training accuracy 0
step 18, training accuracy 0
step 19, training accuracy 0
step 20, training accuracy 0
step 21, training accuracy 0
step 22, training accuracy 0
step 23, training accuracy 0
step 24, training accuracy 0
step 25, training accuracy 0
step 26, training accuracy 0
step 27, training accuracy 0
step 28, training accuracy 0
step 29, training accuracy 0
step 30, training accuracy 0
step 31, training accuracy 0
step 32, training accuracy 0
step 33, training accuracy 0
step 34, tr